In [109]:
import numpy as np
from tqdm.auto import tqdm
from time import time
import json, pickle, os, string, kenlm, json
from collections import defaultdict, Counter
from itertools import groupby
import Levenshtein as Lev
import math

In [110]:
def lse(*args):
    """
    Stable log sum exp.
    """
    if all(a == -float('inf') for a in args):
        return -float('inf')
    a_max = max(args)
#     print('--',args)
#     args = [i for i in args] + [-float('inf')]
    lsp = math.log(sum(math.exp(a - a_max)
                      for a in args))
    return a_max + lsp

In [111]:
def wer_(s1, s2):
    b = set(s1.split() + s2.split())
    word2char = dict(zip(b, range(len(b))))
    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]
    
    return Lev.distance(''.join(w1), ''.join(w2))

In [112]:
labels = "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ |${]"

# greedy decoding

In [113]:
def ctc_best_path(out,labels):
    "implements best path decoding as shown by Graves"
    out = [labels[i] for i in np.argmax(out, axis=1) if i!=labels[-1]]
    o = ""
    for i,j in groupby(out):
        o = o + i
    return o.replace("_","")

# beam search decoding

In [114]:
lm_w = kenlm.LanguageModel('/home/hemant/asr_wm/data/ner/pp_without_misc/without_space/4_gram.arpa')

In [166]:
def sort_beam(ptot,k):
    if len(ptot) < k:
        return [i for i in ptot.keys()]
    else:
        dict_ = sorted(dict((v,k) for k,v in ptot.items()).items(),reverse=True)[:k]
        return [i[1] for i in dict_]

    NEG_INF = -float("inf")
def make_new_beam():
    fn = lambda : (NEG_INF, NEG_INF)
    return defaultdict(fn)

In [157]:
def ctc_beam_search(out,labels, prune=0.0001, k=20,alpha=0.01,beta=0,lm=None):
    
    bc_i = 0 # blank/special charatcter index 
    F = out.shape[1]
    dummy_ = np.vstack((np.zeros(F), out))
    out = np.log(np.vstack((np.zeros(F), out)))
    steps = out.shape[0]
#     print(out)
    pb, pnb = defaultdict(Counter), defaultdict(Counter)
    pb[0][''], pnb[0][''] = 0, -float("inf")
    prev_beams = ['']
    for t in tqdm(range(1,steps)):
        pruned_alphabet = [labels[i] for i in np.where(dummy_[t] > prune)[0]]
        for b in prev_beams:
            for c_t in pruned_alphabet:
                index = labels.index(c_t)
                if c_t == "_": #Extending with a blank
                    if pb[t][b] == 0:
                        pb[t][b] = -float('inf')
                    pb[t][b] = lse(pb[t][b],out[t][index]+pb[t-1][b], out[t][index]+pnb[t-1][b])
                else:
                    i_plus = b + c_t
                    if len(b.replace(' ', '')) > 0 and c_t == b[-1]: #Extending with the same character as the last one
                        pnb[t][b] = lse(pnb[t][b],out[t][index]+pnb[t-1][b])
                        pnb[t][i_plus] = lse(pnb[t][i_plus],out[t][index]+pb[t-1][b])
                    else:
                        pnb[t][i_plus] = lse(out[t][index]+pb[t-1][b], out[t][index]+pnb[t-1][b])
                    if i_plus not in prev_beams:
                        pb[t][i_plus] = lse(pb[t][i_plus],out[t][labels.index("_")]+pb[t - 1][i_plus], out[t][labels.index("_")]+ pnb[t - 1][i_plus])
                        pnb[t][i_plus] = lse(pnb[t][i_plus],out[t][index] + pnb[t - 1][i_plus])


        keys = set(pb[t].keys()).union(set(pnb[t].keys()))
        ptot = {i:pb[t][i]+pnb[t][i] for i in keys }
        print(ptot)
#         break
    
        prev_beams = sort_beam(ptot,k)
        
    return prev_beams[0]


In [159]:
ctc_beam_search(dev[0][0],labels,0.01,k=1,lm=lm_w)

/home/hemant/.conda/envs/deep/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


{'': -0.01811906251038002}
{'': 0.6719636527782209}
{'': 1.0750176415632968}
{'': 1.3616819493325867}
{'': 1.5827948482547327}
{'': 1.7620330217295397}
{'': 1.9111223622646614}
{'': 2.0402034903224306}
{'': 2.1556839708053133}
{'': 2.2595271358441695}
{'': 2.353489404944693}
{'': 2.438402771686339}
{'': 2.515220439967649}
{'': 2.585232038389202}
{'': 2.6497087576239955}
{'': 2.708046831865648}
{'': 2.762442153452545}
{'': 2.8149255253278502}
{'': 2.8641565337217427}
{'': 2.9088075763009464}
{'': 2.9527135528722535}
{'': 2.9944493332081215}
{'': 3.0325884703796637}
{'': 3.0696303247489705}
{'': 3.0708589918792946}
{'': 3.1031284168246747}
{'': 3.068506939603907, 'A': 1.4495007867812728}
{'': -1.1868795507560166, 'I': -1.3994595777340733, 'A': 3.1828568532008847, 'E': -0.8382312975990037}
{'A': 3.1636891432512106}
{'A': 3.1522692276942528, 'AN': 1.2290599535855273}
{'A': -1.392012108295504, 'AN': 3.251435777948242}
{'AND': 3.289990237173283, 'AN': -0.06062445783080225, 'ANE': -0.60685556

"AND CHARGED IF EVER HE MIGHT FINDS SIRR GOAINE] AND  SIR |UWAINNE TO BRING THEM TO THE COUURT  AGAIN AND THEN WERE THEY ALL GLAD AND SO PRAYD THASS  SOR MORE HOUSE TO RIDE WITH THEM TO THE KING'S COURT"

In [120]:
dev[0][1]

"AND CHARGED IF EVER HE MIGHT FIND SIR GAWAINE AND SIR UWAINE TO BRING THEM TO THE COURT AGAIN AND THEN WERE THEY ALL GLAD AND SO PRAYED THEY SIR |MARHAUS] TO RIDE WITH THEM TO THE KING'S COURT"

In [121]:
def error(dummy):
    total_cer, total_wer, num_tokens, num_chars = 0, 0, 0, 0
    for i in tqdm(dummy):
        transcript, reference = i[0], i[1]
        wer_inst = wer_(transcript,reference)
        total_wer += wer_inst
        num_tokens += len(reference.split(' '))\


    wer = (float(total_wer) / num_tokens)*100
    print('Test Summary \t'
        'Average WER {wer:.3f}\t'.format(wer=wer))

In [122]:
def greedy(dummy,p=False):
    d = []
    for i in tqdm(dummy):
        out, reference = i[0], i[1]
        gred_txt = ctc_best_path(out,labels)
        if p: 
            print('transcript: '.upper(),gred_txt)
            print('reference: '.upper(),reference,'\n')
        d.append([gred_txt,reference])
    error(d)

In [123]:
def beam(dummy,labels,prune,beam_width,p=False):
    d = []
    for i in tqdm(dummy):
        out, reference = i[0], i[1]
        beam_txt = ctc_beam_search(out,labels,prune,beam_width)
        if p: 
            print('transcript: '.upper(),beam_txt)
            print('reference: '.upper(),reference,'\n')
        d.append([beam_txt,reference])
    error(d)

In [124]:
pr = True

In [130]:
dev = np.load('/home/hemant/asr_wm/data/ner/pp_without_misc/without_space/dev.npy',allow_pickle=True)
greedy(dev[:1],pr)

TRANSCRIPT:  AND CHARGED IF EVER HE MIGHT FIND SIR GOANE AND SIR |UWANE TO BRING THEM TO THE COURT AGAIN AND THEN WERE THEY ALL GLAD AND SO PRAY TAS OR MOREHOUSE TO RIDE WITH THEM TO THE KING'S COURT
REFERENCE:  AND CHARGED IF EVER HE MIGHT FIND SIR GAWAINE AND SIR UWAINE TO BRING THEM TO THE COURT AGAIN AND THEN WERE THEY ALL GLAD AND SO PRAYED THEY SIR |MARHAUS] TO RIDE WITH THEM TO THE KING'S COURT 





Test Summary 	Average WER 15.385	


In [140]:
beam(dev[:1],labels,0.001,10,pr)

/home/hemant/.conda/envs/deep/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  



TRANSCRIPT:  AND CHARGED IF EVER HE MIGHT FINDS SIR GOAINE] AND SIR |UWAINE TO BRING THEM TO THE COURT AGAIN AND THEN WERE THEY ALL GLAD AND SO PRAYD DHAS  OR MORE HOUSE TO RIDE WITH THEM TO THE KING'S COURT
REFERENCE:  AND CHARGED IF EVER HE MIGHT FIND SIR GAWAINE AND SIR UWAINE TO BRING THEM TO THE COURT AGAIN AND THEN WERE THEY ALL GLAD AND SO PRAYED THEY SIR |MARHAUS] TO RIDE WITH THEM TO THE KING'S COURT 





Test Summary 	Average WER 20.513	
